In [1]:
#--------------IMPORTS AND GLOBALS---------------------

# installs two python modules required
!pip install faker
!pip install ordered-set

# required imports
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import calendar
import math
import numpy as np
import random
from faker import Faker
import re
import datetime
from dateutil.relativedelta import *
import inflect
import torch
import torch.nn as nn
from ordered_set import OrderedSet
import time
import math


# global initialisations
fake = Faker() # initialises faker module for date generation
inf = inflect.engine() # initialises inflect for ordinal number conversion
MAX_LENGTH = 45 # sets max length of input sequences to 45
LEARNING_RATE = 0.01
TEACHER_FORCING = True

# if wanting to both train and validate model on generated data, set TRAINING to true
# otherwise only validation on model will be performed 
TRAINING = False

     |████████████████████████████████| 1.1MB 4.9MB/s 
  Created wheel for ordered-set: filename=ordered_set-4.0.2-py2.py3-none-any.whl size=8209 sha256=c3196e773710e854e570dda8ecada05f180b12bb16163b03836e6d521c818e4c
  Stored in directory: /root/.cache/pip/wheels/e1/c6/9b/651d8a21d59b51a75ab9c070838f9231b8126421bc0569af47
Successfully built ordered-set


In [2]:
#-----------------DICTIONARY-----------------

# create dictionary of characters 
vocab = OrderedSet() # uses ordered set to maintain order within set
vocab.add('<SOS>') # adds start of sentence token
vocab.add('<EOS>') # adds end of sentence token

myfile = open('alphabet.txt', encoding="utf8")  # opens alphabet txt file
for line in myfile:
    for word in line:
        vocab.update(word) # adds each character from alphabet file to the vocab set
myfile.close()

ix2word = dict(enumerate(vocab)) # creates dictionary of vocab set, an index for each word in the vocab

ix2word.update({len(ix2word): '<UNK>'}) # adds unknown character token

word2ix = {v: k for k, v in ix2word.items()} # creates opposite word to index dictionary

In [3]:
#----------DATE GENERATION FUNCTIONS------------------

# functions for generating and retrieving parts of input temporal expression sentences 

# takes randomly generated input number to determine form of day of week returned 
def getDayOfWeek(r):
    # uses faker method day_of_week() to obtain a random day of the week
    if r == 0: return fake.day_of_week() + " " # full - Monday
    elif r == 1: return fake.day_of_week()[0:3] + " " # shortened - Mon
    elif r == 2: return fake.day_of_week()[0:3] + '. ' # abbreviated - Mon.
    else: return "" # returns nothing
  
# returns random time of day w/ optional timezone
def getTime(r):
    if r == 0:
        r = random.randint(1, 12) # generates random hour in 12 hour clock
        r2 = random.randint(0,3) # random number generated to determine the form of the returned time
        if r2 == 0: return str(r) + " am" + getTimeZone()  # 12 am PT
        elif r2 == 1: return str(r) + " pm" + getTimeZone() # 12 pm
        elif r2 == 2: return str(r) + " AM" + getTimeZone() # 12 AM
        elif r2 == 3: return str(r) + " PM" + getTimeZone() # 12 PM ET
    else: return ""
    
# returns a US timezone or no timezone
def getTimeZone():
    r = random.randint(0,14) # generates random number to detemine which, if any, timezone is returned, weighted to mostly not return one
    if r == 0: return " PT" # pacific
    elif r == 1: return " MT" # mountain
    elif r == 2: return " CT" # central
    elif r == 3: return " ET" # eastern
    else: return "" # blank
    
# returns a numerical day in the month using faker method day_of_month()
def getDay():
    return fake.day_of_month() # DD

# separate method to generate various forms of days, takes input numerical day and random number to determine form
# a tuple of the new day form and the original input day form is returned so YYYY-MM-DD label can be formed too 
def convertDay(d,r):
    if r == 0: return (d,d) # 05
    elif r == 1: return (d.lstrip("0"),d) # 5
    elif r == 2: return (inf.ordinal(int(d.lstrip("0"))),d) # 5th
    elif r == 3: return (inf.number_to_words(inf.ordinal(int(d.lstrip("0")))).capitalize(),d) # fifth
    
# returns random month tuple of the year in various forms with faker month()
def getMonth(r):
    m = fake.month() # generates month in form MM

    if r == 0: return (m, m) # 01
    elif r == 1: return (m.lstrip("0"), m) # 1, 01
    elif r == 2: return (calendar.month_name[int(m)], m) # January , 01
    elif r == 3: return (calendar.month_abbr[int(m)], m) # Jan, 01
    #elif r == 4: return (calendar.month_abbr[int(m)] + '.', m) # Jan., 01

# returns random year tuple with faker year(), begins with 1970
def getYear(r):
    y = fake.year() # generates year in form YYYY

    if r == 0: return (y,y) # 1999
    elif r == 1: return (y[-2:], y) # 99

# returns a random holiday of listed holidays, as well as their MM-DD form for label
def getHoliday(r):
    if r == 0: return ("Christmas", "12-25")
    elif r == 1: return ("Halloween", "10-31")
    elif r == 2: return ("New Years Day", "01-01")
    elif r == 3: return ("New Years Eve", "12-31")
    elif r == 4: return ("Boxing Day", "12-26")
    elif r == 5: return ("Valentines Day", "02-14")
    elif r == 6: return ("April Fools Day", "04-01")
    elif r == 7: return ("St Patricks Day", "03-17")
    elif r == 8: return ("All Saints Day", "11-01")
    elif r == 9: return ("year-end", "12-31")
    elif r == 10: return ("Independence Day", "07-04")

# returns random delimeter between parts of numerical date formats
def getDivider(r):
    if r == 0: return '/'
    elif r == 1: return '-'
    elif r == 2: return '.'
    elif r == 3: return ''
    elif r == 4: return ' '

# returns random relative expression interval word
def getRelativeInterval(r):
    # 0
    if r == 0: return "today"
    elif r == 1: return "now"
    elif r == 2: return "this morning"
    elif r == 3: return "this afternoon"
    elif r == 4: return "this evening"
    elif r == 5: return "tonight"
    elif r == 6: return getTime(0)
    # +1
    elif r == 7: return "tomorrow"
    elif r == 8: return "next "
    elif r == 9: return " later"
    elif r == 10: return " after"
    # -1
    elif r == 11: return "yesterday"
    elif r == 12: return "last "
    elif r == 13: return "previous "
    elif r == 14: return " before"
    elif r == 15: return " earlier"
    # -2
    elif r == 16: return "day before yesterday"
    # +2
    elif r == 17: return "day after tomorrow"

# returns random relative expression words
def getRelativeWord(r):
    if r == 0: return "next year"
    elif r == 1: return "last year" 
    elif r == 13: return "this"
    elif r == 14: return "next"
    elif r == 15: return "coming"
    elif r == 16: return "last"
    elif r == 17: return "previous"
    elif r == 18: return "start of the "
    elif r == 19: return "beginning of the "
    elif r == 20: return "end of the "

# returns random measurement of time    
def getMeasurement(r):
    if r == 0: return " days"
    elif r == 1: return " weeks"
    elif r == 2: return " months"
    elif r == 3: return " years"

# returns random relative words
def getRelativeMod(r):
    if r == 0: return " ago"
    elif r == 1: return " from now"
    elif r == 2: return "in "
    elif r == 3: return "last"
    elif r == 4: return "next"
    

In [4]:
#--------- HELPER FUNCTIONS--------------

# uses datetime object to verify legimate dates have been generated, recurses and decreases day value until legit date formed
def verifyDate(d, m): # takes day and month
    date = m + "-" + d # forms date in MM-DD
    try:
        datetimeobj = datetime.datetime.strptime(date, "%m-%d").date() # creates datetime object of MM-DD
        return (d) # returns the valid day 
    except ValueError: # checks for value error
        return verifyDate(str(int(d)-1), m) # recurses and decreases day value by 1
        
# allows adding and subtracting of days/weeks/months/years from dates using datetime object functionality
def dateArithmetic(d, m, y, change, measurement): # takes day, month, year, numerical change value, measurement of time to change
    datetime_date = datetime.datetime.strptime(y + "-" + m + "-" + d, "%Y-%m-%d").date() # creates datetime object

    # if statement block checks which measurement is to be used, and increases/decreases off change value
    if measurement == "d":
        diff = datetime.timedelta(days=change)
    elif measurement == "w":
        diff = datetime.timedelta(weeks=change)
    elif measurement == "m":
        diff = relativedelta(months=change)
    elif measurement == "y":
        diff = relativedelta(years=change)
    
    return ((datetime_date + diff).strftime("%Y-%m-%d")) # converts new datetime back to string and returns
    
# returns closest year to the current date
# IS NOT USED
def getClosestYear(nd, nm, y, rd):
    ref = datetime.datetime.strptime(rd[1:11], "%Y-%m-%d").date()
    y = str(int(y)-2)
    mn = 1000
    
    for i in range(3):
        y = str(int(y)+1)
        date = datetime.datetime.strptime(y+"-"+nm+"-"+nd, "%Y-%m-%d").date()
        diff = abs((date - ref).days)
        if diff < mn:
            mn = abs((date - ref).days)
            mn_year = y
            
    return mn_year

# retrieves yyyy-mm-dd form of the start/end of a month/year
def getStartEnd(m, y, measurement, start): # takes month value, year value, month/year measurement, and boolean meaning start or end
    if measurement == "month":
        if start:
            return y + "-" + m + "-"+ "01"
        else:
            d = calendar.monthrange(int(y), int(m))[1] # uses calendar module monthrange method to obtain the final day of the input month
            return y + "-" + m + "-" + str(d)
    elif measurement == "year":
        if start:
            return y + "-" + "01-01"
        else:
            return y + "-" + "12-31"

# pads input sequences with leading zeroes to a specified length
def padInput(data, seq_len):
    padded = np.zeros((len(data), seq_len),dtype=int)
    for ind, seq in enumerate(data):
        if len(seq) != 0:
            padded[ind, -len(seq):] = np.array(seq)[:seq_len]
    return padded

In [5]:
#------------GENERATE DATA-------------

# function takes max_range - total number of examples to generate, and ratios - list of how many examples to generate per subcategory
def genData(max_range, ratios):
    data = [] # data list
    labels = [] # labels list
    count = 0 # count incremented at each iteration
    expr_types = [0,0,0,0,0,0,0,0,0,0] # list holding the number of expressions generated per subclass

    for x in range(0, max_range): # iterates max_range times to generate max_range expressions
    
        # obtains a month, day, year value calling previously defined functions to create a reference date
        month = getMonth(random.randint(0,3))
        day = verifyDate(getDay(), month[1])
        year = getYear(0)
        ref_date = "<" + year[1] + "-" + month[1] + "-" + day + ">" # creates ref date in form <YYYY-MM-DD>

        # ---------ABSOLUTE EXPRESSIONS--------------
        # HOLIDAY EXPRESSIONS 
        if count < ratios[0]: # creates holiday expressions up to a certain point

            # obtains random holiday and year, creates date and label

            hol = getHoliday(random.randint(0,10))
            year = getYear(random.randint(0,1))
            div = getDivider(4)
            date = hol[0] + div + year[0] + " " + ref_date
            label = year[1] + '-' + hol[1]
            expr_types[0]+=1

        # US DATE EXPRESSIONS
        elif count < ratios[1]:
            # generates random month
            month = getMonth(random.randint(0,3))
                
            if bool(re.search(r'\d', month[0])): # if numerical month
                year = getYear(random.randint(0,1)) # gets year (not including apostrophe eg '19)
                day = convertDay(verifyDate(getDay(), month[1]), random.randint(0,1))
                div = getDivider(random.randint(0,1)) # gets non space divider
                dow = getDayOfWeek(6) # gets blank day of week
                
                
            else: # textual month
                year = getYear(random.randint(0,1))
                day = convertDay(verifyDate(getDay(), month[1]), random.randint(0,3))
                div = getDivider(4) # get space divider
                dow = getDayOfWeek(random.randint(0,6)) # gets day of week (can be blank)
            
            date = month[0] + div + day[0] + div + year[0] + " " + ref_date # us date format
            label = year[1] + "-" + month[1] + "-" + day[1]
            expr_types[1]+=1
            
        # DOCUMENT CREATION CODE 
        elif count < ratios[2]: 
            year = getYear(random.randint(0,1)) # gets year
            month = getMonth(0)
            day = convertDay(verifyDate(getDay(), month[1]), 0)
                
            r = random.randint(0,1)
            if r == 0:
                date = year[1] + month[1] + day[1] + " " + ref_date # 19980505 format
            else:
                date = year[1][-2:] + month[1] + day[1] + " " + ref_date # 980505 format
                
            label = year[1] + "-" + month[1] + "-" + day[1]
            expr_types[2]+=1       
            
        # ---------RELATIVE EXPRESSIONS--------------
        # US DATE
        elif count < ratios[3]:
                
            new_month = getMonth(random.randint(0,3))
                
            if bool(re.search(r'\d', new_month[0])): # if numerical month
                div = getDivider(random.randint(0,1)) # gets non space divider
                new_day = convertDay(verifyDate(getDay(), new_month[1]), random.randint(0,1))
                dow = getDayOfWeek(6) # gets blank day of week
            else: # textual month
                div = getDivider(4) # get space divider
                new_day = convertDay(verifyDate(getDay(), new_month[1]), random.randint(0,3))
                dow = getDayOfWeek(random.randint(0,6)) # gets day of week (can be blank)
                
            r = random.randint(0,1)
            date = new_month[0] + div + new_day[0] + " " + ref_date # us date format
                
            label = year[1] + "-" + new_month[1] + "-" + new_day[1]
            expr_types[3]+=1
                
        # HOLIDAYS
        elif count < ratios[4]:
                
            hol = getHoliday(random.randint(0,10))
            date = hol[0] + " " + ref_date
            label = year[1] + '-' + hol[1]
            expr_types[4]+=1
                
        # MISCELLANEOUS
        elif count < ratios[5]:
            r = random.randint(0,3)
            measure = getMeasurement(r)[1:-1]
            r = random.randint(0,17)
            when = getRelativeInterval(r)
            if r <= 6: # same day
                date = when + " " + ref_date
                label = year[1] + "-" + month[1] + "-" + day
            elif r <= 10 and r > 6: # day after
                if r == 7: 
                    date = when + " " + ref_date
                    measure = "day"
                elif r == 8: date = when + measure + " " + ref_date
                else: date = measure + when + " " + ref_date
                label = dateArithmetic(day, month[1], year[1], 1, measure[:1])
            elif r <= 15 and r > 10: # day before
                if r == 11: 
                    date = when + " " + ref_date
                    measure = "day"
                if r == 12:
                    if measure == "day": date = when + "night" + " " + ref_date 
                    else: date = when + measure + " " + ref_date
                elif r == 13: date = when + measure + " " + ref_date
                elif r == 14 or r == 15: date = measure + when + " " + ref_date
                label = dateArithmetic(day, month[1], year[1], -1, measure[:1])
            elif r == 16: # day before yesterday
                date = when + " " + ref_date
                label = dateArithmetic(day, month[1], year[1], -2, "d")
            elif r == 17: #day after tomorrow
                date = when + " " + ref_date
                label = dateArithmetic(day, month[1], year[1], 2, "d")
            expr_types[5]+=1
                
        # US DATE
        elif count < ratios[6]:
            r = random.randint(0,1)
            dow = getDayOfWeek(random.randint(0,6))
            div = getDivider(4)
            new_month = getMonth(random.randint(2,3))
            new_day = verifyDate(getDay(), month[1])
            if r == 0:
                date = new_month[0] + div +  new_day + " " + getRelativeWord(0) + " " + ref_date # US next year
                label = str(int(year[1]) + 1) + "-" + new_month[1] + "-" + new_day
            elif r == 1:
                date = new_month[0] + div +  new_day + " " + getRelativeWord(1) + " " + ref_date # US last year
                label = str(int(year[1]) - 1) + "-" + new_month[1] + "-" + new_day
            expr_types[6]+=1
        
        # HOLIDAYS
        elif count < ratios[7]:
            
            hol = getHoliday(random.randint(0,10))
            r = random.randint(13,17)
            when = getRelativeWord(r)
            
            date = when + " " + hol[0] + " " + ref_date
            label = year[1] + "-" + hol[1]
            
            if r < 16: # future (next/coming/this)
                if (ref_date[1:11] > label):
                    new_year = str(int(year[1]) + 1) # increases year if future holiday is in the following year
                    label = new_year + "-" + hol[1] # updates label
            else: # past (previous/last)
                if (ref_date[1:11] < label):
                    new_year = str(int(year[1]) -1) # decreases year if past holiday is in previous year
                    label = new_year + "-" + hol[1] # updates label
            expr_types[7]+=1   
                
        # DATE ARITHMETIC
        elif count < ratios[8]:
            
            r = random.randint(0,3)
                
            measurement = getMeasurement(r)
            measurement_short = measurement[1:2]
                
            if r == 0:
                change = random.randint(2,10) # 2-10 days
            elif r == 1:
                change = random.randint(1,6) # 1-6 weeks
            elif r == 2:
                change = random.randint(1,6) # 1-6 months
            elif r == 3:
                change = random.randint(1,5) # 1-5 years
                
            if change == 1:
                measurement = measurement[:-1] # makes measurement word singular
            
            num_word = str(change)
            
            r = random.randint(0,2)
            
            if  r == 0:
                date = num_word + measurement + getRelativeMod(0) + " " + ref_date # x days ago
                label = dateArithmetic(day, month[1], year[1], -change, measurement_short)
            elif r == 1:
                date = num_word + measurement + getRelativeMod(1) + " " + ref_date # x days from now
                label = dateArithmetic(day, month[1], year[1], change, measurement_short)
            elif r == 2:
                date = getRelativeMod(2) + num_word + measurement + " " + ref_date # in x days
                label = dateArithmetic(day, month[1], year[1], change, measurement_short)
            expr_types[8]+=1
                
        # START/END
        elif count < ratios[9]:
                
            r = random.randint(18,21)
            if r == 20 or r == 21:
                r = 20
                start = False 
            else: # 1/2 chance of start/beginning of
                start = True
            phrase = getRelativeWord(r)
                
            r = random.randint(0,13)
            if r == 0:
                measurement = getMeasurement(3)[1:-1] # gets year
            else:
                measurement = getMeasurement(2)[1:-1] # gets month
                
            if measurement == "month":
                r = random.randint(0,12)
                if r != 0: # obtains name of month as opposed to "month"
                    new_month = getMonth(2)
                    date = phrase[:-4] + new_month[0] + " " + ref_date
                    label = getStartEnd(new_month[1], year[1], measurement, start)
                else:
                    date = phrase + measurement + " " + ref_date
                    label = getStartEnd(month[1], year[1], measurement, start)
            else:
                date = phrase + measurement + " " + ref_date
                label = getStartEnd(month[1], year[1], measurement, start)
            expr_types[9]+=1
                
        # adds date to data list
        data.append(date)
        # adds label to label list
        labels.append(label)
        
        # increments count
        count+=1    

    print("NUM EXPRESSIONS PER SUBCLASS: ",  expr_types)
    shuffled_list = list(zip(data,labels)) # shuffles data and label lists
    random.shuffle(shuffled_list)
    return zip(*shuffled_list) # returns data and labels

In [6]:
#------------CONVERTS DATA TO TENSOR FORM-----------------

# takes the training and testing data and converts the lists into tensors 
def dataToTensor(train_data, train_labels, test_data, test_labels, validation):
    
    # creates copies of the training and testing data
    temp_train_data = train_data.copy()
    temp_train_labels = train_labels.copy()
    temp_test_data = test_data.copy()
    temp_test_labels = test_labels.copy()

    # converts the data to numerical form off the word2ix dictionary

    for i, sentence in enumerate(temp_train_data):
        temp_train_data[i] = [word2ix[word] if word in word2ix else 0 for word in sentence]
        temp_train_data[i].append(1)

    for i, sentence in enumerate(temp_test_data):
        temp_test_data[i] = [word2ix[word] if word in word2ix else 0 for word in sentence]
        temp_test_data[i].append(1)
    
    for i, sentence in enumerate(temp_train_labels):
        temp_train_labels[i] = [word2ix[word] if word in word2ix else 0 for word in sentence]
        temp_train_labels[i].append(1)
    
    for i, sentence in enumerate(temp_test_labels):
        temp_test_labels[i] = [word2ix[word] if word in word2ix else 0 for word in sentence]
        temp_test_labels[i].append(1)


    seq_len = 45  # The length that the sentences will be padded/shortened to
    
    # pads train and test data and labels
    temp_train_data = padInput(temp_train_data, seq_len)
    temp_test_data = padInput(temp_test_data, seq_len)    

    if validation:
        temp_train_labels = np.array(temp_train_labels)
        temp_test_labels = np.array(temp_test_labels)
    else:
        temp_train_labels = padInput(temp_train_labels, 11)
        temp_test_labels = padInput(temp_test_labels, 11)     # 11 length of reference date <yyyy-mm-dd> 

    # tensor lists
    ntrain_data = []
    ntest_data = []

    # converts training and testing data into tensors and appends to lists
    for i in range(len(temp_train_data)):
        ntrain_data.append(((torch.from_numpy(temp_train_data[i]).long()).view(-1, 1),(torch.from_numpy(temp_train_labels[i]).long()).view(-1, 1)))
    for i in range(len(temp_test_data)):
        ntest_data.append(((torch.from_numpy(temp_test_data[i]).long()).view(-1, 1),(torch.from_numpy(temp_test_labels[i]).long()).view(-1, 1)))

    return ntrain_data, ntest_data # returns tensor lists

In [7]:
#----------ENCODER AND DECODER------------

# encoder class
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size) # embedding layer
        self.gru = nn.GRU(hidden_size, hidden_size) # gru layer

    def forward(self, input, hidden): # foward pass
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self): # initialises hidden state
        return torch.zeros(1, 1, self.hidden_size, device=device) # zeroes initial hidden state

# decoder class
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size) # embedding layer
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length) # attention layer
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p) # dropout layer
        self.gru = nn.GRU(self.hidden_size, self.hidden_size) # gru layer
        self.out = nn.Linear(self.hidden_size, self.output_size) # final linear layer

    def forward(self, input, hidden, encoder_outputs): # forward pass
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device) # zeroes initial hidden state

In [8]:
# ---------------TRAIN PHASE FUNCTIONS-------------

# two functions to print the time since the execution of the training began

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [9]:
#---------------TRAINING METHOD-----------------

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden() # initiliases first hidden state

    # zeroes gradients on optimisers
    encoder_optimizer.zero_grad() 
    decoder_optimizer.zero_grad()

    # initialises inputs and outputs
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for i in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
        encoder_outputs[i] = encoder_output[0, 0]
    
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden


    if TEACHER_FORCING: # teacher forcing has target label used as next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else: # without teacher forcing, predictions used as next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [10]:
#-----------------------TRAINING LOOP------------------

def trainingPhase(encoder, decoder, max, print_every=1000, plot_every=100, learning_rate=LEARNING_RATE):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0 
 
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    # gets list of pairs of data and labels in tensored training data
    training_pairs = [(ntrain_data[i][0], ntrain_data[i][1]) for i in range(max)]

    criterion = nn.NLLLoss()

    # loop through each example in training data
    for i in range(1, max + 1):
        # retrieves new pair from list
        training_pair = training_pairs[i - 1]
        # sets input and targets from pair
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        # calls training method passing the input and target
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, i / max), i, i / max * 100, print_loss_avg))

        if i % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


In [11]:
#---------------------TEST METHOD-------------------------------

def test(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = sentence
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                break
            else:
                decoded_words.append(ix2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [12]:
#---------------------TESTING LOOP-------------------------------
def testPhase(encoder, decoder, n):
    corr = 0 # int to store how many correct predictions made
    print("NUMBER OF EXAMPLES: ", n)

    for i in range(n): # begins loop
        if i % (n/10) == 0:
            print(f"{(i/len(test_data))*100}% complete.")
        
        pair = [(ntest_data[i][0], ntest_data[i][1])] # gets pair of example and label 

        # constructs string of target tensor with ix2word dict
        outp = []
        for j in range((len(pair[0][1]))-1):
            if pair[0][1][j].item() != 0:
                outp.append(ix2word[pair[0][1][j].item()])

        targ_str = "".join(outp)
        
        # passes inp tensor to test function
        output_words, attentions = test(encoder, decoder, pair[0][0])
        output_sentence = ''.join(output_words) # constructs sentence from output words
        
        if output_sentence == targ_str: # if prediction equivalent to target
            corr+=1 # correct prediction made

    print(f"{100}% complete.")
    print("")
    print("correct: ", corr, " of ", n)
    print(f"acc: {(corr/n)*100:.4f}%")

In [13]:
# RUN CELL IF USING GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
#--------------------GENERATE DATA-----------------------

max_range =  125000 # total number of examples
MAX_LENGTH = 45
SOS_token = 0
EOS_token = 1 
train_perc = int(max_range * 0.8) # number of training examples
test_perc =  max_range - train_perc  # number of validation exmamples

# ratios list holds the indexes at which a new subcategory of expression should be generated
ratios = [train_perc/10, 2*train_perc/10, 3*train_perc/10, 4*train_perc/10, 5*train_perc/10, 6*train_perc/10, 7*train_perc/10, 8*train_perc/10, 9*train_perc/10, train_perc]
train_data, train_labels = genData(train_perc, ratios) # generates training data and labels
print("LENGTH OF TRAINING LISTS: ", len(train_data))

ratios = [test_perc/10, 2*test_perc/10, 3*test_perc/10, 4*test_perc/10, 5*test_perc/10, 6*test_perc/10, 7*test_perc/10, 8*test_perc/10, 9*test_perc/10, test_perc]
test_data, test_labels = genData(test_perc, ratios)
print("LENGTH OF TESTING LISTS: ", len(test_data))

# converts data into tensors
train_data = list(train_data)
train_labels = list(train_labels)
test_data = list(test_data)
test_labels = list(test_labels)
ntrain_data, ntest_data = dataToTensor(train_data, train_labels, test_data, test_labels, True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NUM EXPRESSIONS PER SUBCLASS:  [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]
LENGTH OF TRAINING LISTS:  100000
NUM EXPRESSIONS PER SUBCLASS:  [2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500]
LENGTH OF TESTING LISTS:  25000


In [15]:
#-----------------TRAINING AND TESTING-----------------

# initialise encoder and decoder classes
hidden_size = 256
enc = Encoder(len(ix2word), hidden_size).to(device)
dec = Decoder(hidden_size, len(ix2word), dropout_p=0.1).to(device)

print("------LOADING PREVIOUS MODEL------")

# IF USING GOOGLE DRIVE USE CURRENT PATHS
# OTHERWISE UPLOAD MODELS TO COLAB LOCAL STORAGE AND CHANGE PATHS
path = F"/content/gdrive/My Drive/models/tf/encoder.pt" # google drive path
#path = 'encoder.pt' # local path
enc.load_state_dict(torch.load(path))
path = F"/content/gdrive/My Drive/models/tf/decoder.pt" # google drive path
#path = 'decoder.pt' # local path
dec.load_state_dict(torch.load(path))
print("Successfully loaded encoder and decoder.")
print("")

if TRAINING: # trains model if training boolean set to true
    print("------MODEL TRAINING------")
    trainIters(enc, dec, int(train_perc), print_every=(len(train_data)/50)) # calls training function
    print("")

print("------GENERAL MODEL VALIDATION------")
testPhase(enc, dec, len(ntest_data)) # tests model
print("")

if TRAINING:
    print("------MODEL SAVING------")
    model_save_name = 'new_encoder.pt'
    path = F"/content/gdrive/My Drive/models/tf/{model_save_name}" # google drive path
    #path = 'new_encoder.pt' # local path
    torch.save(enc.state_dict(), path)

    model_save_name = 'new_decoder.pt'
    path = F"/content/gdrive/My Drive/models/tf/{model_save_name}" # google drive path
    #path = 'new_decoder.pt' # local path
    torch.save(dec.state_dict(), path)
    print("Successfully saved encoder and decoder.")
    print("")

print("------GENERATING VALIDATION SETS PER SUBCATEGORY------")
test_perc = 10000 # 10k examples per subcategory
test_data_arr = []
test_labels_arr = []
for i in range(0,10): # generates data 
    if i < 1:
        ratios[2] = test_perc + 1
    ratios[i] = test_perc
    test_data, test_labels = genData(test_perc, ratios)
    test_data = list(test_data)
    test_labels = list(test_labels)
    test_data_arr.append(test_data)
    test_labels_arr.append(test_labels)
    ratios[i] = 0
print("")

print("------MODEL VALIDATION ON EACH SUBCLASS------")
for i in range(0, 10):
  test_data = test_data_arr[i]
  test_labels = test_labels_arr[i]
  ntrain_data, ntest_data = dataToTensor(train_data, train_labels, test_data, test_labels, True)

  print("EXAMPLES OF EXPRESSION AND LABEL: ", test_data[1]," | ",test_labels[1])
  testPhase(enc, dec, len(ntest_data))
  print("")

------LOADING PREVIOUS MODEL------
Successfully loaded encoder and decoder.

------GENERAL MODEL VALIDATION------
NUMBER OF EXAMPLES:  25000
0.0% complete.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100% complete.

correct:  23595  of  25000
acc: 94.3800%

------GENERATING VALIDATION SETS PER SUBCATEGORY------
NUM EXPRESSIONS PER SUBCLASS:  [10000, 0, 0, 0, 0, 0, 0, 0, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 10000, 0, 0, 0, 0, 0, 0, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 0, 10000, 0, 0, 0, 0, 0, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 0, 0, 10000, 0, 0, 0, 0, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 0, 0, 0, 10000, 0, 0, 0, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 0, 0, 0, 0, 10000, 0, 0, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 0, 0, 0, 0, 0, 10000, 0, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 0, 0, 0, 0, 0, 0, 10000, 0, 0]
NUM EXPRESSIONS PER SUBCLASS:  [0, 0, 0, 0, 0, 0, 0, 0, 100

In [22]:
#----------------EVALUATE MODEL ON UNSEEN TEST DATA----------------

enc = Encoder(len(ix2word), hidden_size).to(device)
dec = Decoder(hidden_size, len(ix2word), dropout_p=0.1).to(device)

print("------PREVIOUS MODEL LOADING------")
path = F"/content/gdrive/My Drive/models/tf/encoder.pt" # google drive path
#path = 'encoder.pt' # local path
enc.load_state_dict(torch.load(path))
path = F"/content/gdrive/My Drive/models/tf/decoder.pt" # google drive path
#path = 'decoder.pt' # local path
dec.load_state_dict(torch.load(path))
print("Successfully loaded encoder and decoder.")
print("")


# initialises data and labels lists
data = []
labels = []

# opens and reads data, refdates and labels files into lists

path = "timebank_data.txt"
myfile = open(path, encoding="utf8") # opens file
for line in myfile:
    data.append(line[:-1])
myfile.close() 


path = "timebank_dates.txt"
myfile = open(path, encoding="utf8") # opens file
count = 0
for line in myfile:
    data[count] = data[count] + "<" + line[:-1] + ">" # formats file content into ref date form and appends to each expression in data list
    count +=1
myfile.close()


path = "timebank_labels.txt"
myfile = open(path, encoding="utf8") # opens file
for line in myfile:
    labels.append(line[:-1])
myfile.close()


train_data = []
train_labels = []
test_data = data
test_labels = labels
# converts test data into tensors
ntrain_data, ntest_data = dataToTensor(train_data, train_labels, test_data, test_labels, False)

# model testing
print("------MODEL TESTING ON TIMEBANK------")
testPhase(enc, dec, len(ntest_data))

------PREVIOUS MODEL LOADING------
Successfully loaded encoder and decoder.

------MODEL TESTING ON TIMEBANK------
NUMBER OF EXAMPLES:  366
0.0% complete.
100% complete.

correct:  348  of  366
acc: 95.0820%
